<h1> Melakukan Fine-tuning pada Model Whisper (Small) untuk implementasi Automatic Speech Recognition bahasa Indonesia</h1>

Penelitian ini dilakukan untuk membangun sistem pengenalan ucapan otomatis (ASR) pada bahasa Indonesia. Model yang diadopsi menggunakan Whisper (Small) yang dibangun oleh OpenAI. Library yang digunakan dalam penelitian ini adalah Huggingface Tranformers, PyTorch, dan Jiwer.

Keywords: **ASR, Bahasa Indonesia, Whisper, Fine-tuning**

## Menginstall dan mengupdate beberapa modul dan library
Agar sistem dapat berjalan secara kompatibel, kita perlu memastikan setiap modul dan library di lingkungan kerja kita up-to-date. Beberapa modul dan library yang perlu diupdate adalah pip, huggingface library, accelerate, jiwer, tensorboard, dan gradio.

In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

## Masuk ke Huggingface Hub untuk mengakses dataset dan model

Jika belum membuat akun Huggingface, silakan membuatnya dengan mengunjungi [https://huggingface.co](). Setelah anda membuat akun, silakan buka profil Anda dengan mengklik ikon foto profil dan buka bagian Access Tokens. Buat token akses baru lalu copy code tersebut saat blok ini dijalankan.

**Perhatian!** Terdapat dua mode dari token akses: read dan write. Jika Anda ingin mengupload model atau dataset anda ke Huggingface Hub, pastikan untuk membuat token akses dengan mode **write**.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Buat Konstruktor Huggingface Dataset dan Muat Dataset Audio Corpus Bahasa Indonesia dengan Common Voice 16.1

Semenjak dataset common voice 16.1 bersifat open-source, Anda dapat menggunakannya dengan menyisipkan alamat repository Huggingface Hub dari dataset ini dengan metode `load_dataset` dari modul `datasets`. Dataset Common Voice merupakan dataset audio dan transkripsi multibahasa, sehingga Anda hanya ingin mengambil subset yang berbahasa Indonesia. Anda dapat melakukannya dengan menggunakan syntax

```python
load_dataset("mozilla-foundation/common_voice_16_1", "id", split="<tentukan split anda disini>", use_auth_token=True)
```

Perhatikan bahwa `id` adalah kode yang menunjukan subset untuk bahasa Indonesia.

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
common_voice['train'] = load_dataset("mozilla-foundation/common_voice_16_1", "id", split="train+validation", use_auth_token=True)
common_voice['test'] = load_dataset("mozilla-foundation/common_voice_16_1", "id", split="test", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You 

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4969it [00:00, 94390.84it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3341it [00:00, 115451.61it/s]


...


Reading metadata...: 3643it [00:00, 133034.84it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 29031it [00:00, 148233.03it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2597it [00:00, 91630.77it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 8310
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 3643
    })
})


Kita dapat melihat bahwa dataset ini terdiri dari beberapa komponen yakni `client_id, path, sentence, up_votes,` dan seterusnya.

<br>

Kita hanya menginginkan beberapa komponen seperti `audio` dan `sentence` untuk implementasi ASR, sehingga mari hapus komponen yang tidak relevan :)

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

## Muat Modul Ekstraktor Fitur, Tokenizer, dan Model Whisper (Pretrained Small)

Dataset yang kita muat memiliki format berupa array berdimensi 2×N dimana 2 menunjukan bahwa audio kita memiliki format stereo dan N menunjukkan timestep. Kita perlu mengubah ini menjadi representasi spektogram.

Umumnya, terdapat beberapa tahapan yang perlu dilakukan untuk dapat mengubah representasi gelombang audio menjadi representasi fitur dalam bentuk spektogram, seperti melakukan fast fourier transform, kalkulasi mel filter bank, dan sebagainya.

Kabar baiknya, modul Huggingface Whisper telah menyediakan fungsi yang dapat melakukan ini secara otomatis. Kita hanya perlu mengimpor modul `WhisperFeatureExtractor` dan `WhisperProcessor` untuk menggunakannya sebagai preprocessor data kita.

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Indonesian", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Praproses dapat dilakukan dengan memanfaatkan modul feature extractor dan tokenizer yang telah kita definisikan di atas. Sebagai contoh, mari kita coba melakukan praproses pada salah satu string dari dataset split train di index pertama.

In [ ]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Saya mendengarkan cerita membosankan dari teman saya.
Decoded w/ special:    <|startoftranscript|><|id|><|transcribe|><|notimestamps|>Saya mendengarkan cerita membosankan dari teman saya.<|endoftext|>
Decoded w/out special: Saya mendengarkan cerita membosankan dari teman saya.
Are equal:             True


Amati apa yang dilakukan oleh fungsi `tokenizer` di atas. Modul `tokenizer` mengubah/mengencode transkripsi yang memiliki format string menjadi numerik. Sebagai informasi, Whisper menerapkan algoritma **Byte-pair Encoding** atau biasa disingkat BPE untuk melakukan encoding ini. Teknik ini sebenarnya diadopsi OpenAI dalam modul praproses di GPT-2 dan GPT-3. String yang diencode dengan BPE ini akan digunakan model dalam masa pelatihan.

Untuk melakukan decoding, kita bisa menjalankan metode `.decode` dari tokenizer dan memberikan hasil encoding ke metode ini. Contoh:
```python
tokenizer.decode(<arrays hasil encoding>, skip_special_tokens=False)
```
Perhatikan bahwa ada hal menarik disini, kita melihat argumen `skip_special_tokens` di masukkan ke dalam fungsi decode sebagai parameter. Apa yang sebenarnya terjadi?

Berdasarkan paper resmi dari Whisper, model ini menerima input dengan format khusus; terdapat beberapa token unik sebagai indikator dari teks. Dari sel di atas, Anda dapat melihat bahwa ketika kita memberikan argumen `False` pada `skip_special_tokens`, kita akan mendapatkan format seperti
```
<|startoftranscript|><|id|><|transcribe|><|notimestamps|>Saya mendengarkan cerita membosankan dari teman saya.<|endoftext|>
```
Representasi ini memberikan label tambahan pada teks yang membuat Whisper dapat mempelajari data transkripsi lebih baik. Silakan kunjungi [link ini](https://github.com/openai/whisper/tree/main) untuk deskripsi lebih detail, saya sangat menyarankannya!

Selanjutnya kita bisa memuat model Whisper Small kita, pastikan untuk memasukkan argumen `language="Indonesian"` dan `task="transcribe"` untuk memastikan agar kita memuat pretrained weights dalam mode transkripsi bahasa Indonesia (parameter `task` dapat memiliki 2 opsi, seperti `"transcribe"` untuk transkripsi, dan `"translate"` untuk penerjemahan).

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Indonesian", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/709655cbfa2cb0b3c81672bba7b38c8e4b723fff99a6dcaaf4bc87449a9bb7a9/id_train_0/common_voice_id_25361388.mp3', 'array': array([ 0.00000000e+00, -7.95807864e-13, -1.39266376e-12, ...,
        1.01578926e-05,  2.26661632e-05,  1.37273510e-05]), 'sampling_rate': 48000}, 'sentence': 'Saya mendengarkan cerita membosankan dari teman saya.', 'variant': ''}


## Lakukan Training Pada Model

Ada beberapa hal yang perlu diperhatikan sebelum kita melakukan training pada model ini. Catat bahwa...
- Angka sampling rate audio yang diterima dalam model Whisper adalah 16000
- Perhatikan performa GPU Anda untuk mencegah Out Of Memory Error! Jika Anda mengalami kasus ini, pastikan agar mengatur batch size ke ukuran lebih kecil pada modul trainer.

Ingat bahwa kita menggunakan metrik WER untuk mengukur kualitas performa dari model kita. WER adalah Word Error Rate, metrik ini mengukur seberapa sering model melakukan kesalahan dalam transkripsi. Untuk informasi, formula WER dihitung sebagai berikut.


\begin{align}
\text{WER} = \frac{\text{S} + \text{D} + \text{I}}{\text{N}}
\end{align}

Dengan:
- $\text{S}$ adalah jumlah kata yang disubstitusi (substitutions)
- $\text{D}$ adalah jumlah kata yang dihapus (deletions)
- $\text{I}$ adalah jumlah kata yang disisipkan (insertions)
- $\text{N}$ adalah jumlah total kata dalam teks referensi.

Mari kita lakukan!

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000)) # Cast sampling rate yang awalnya 48000 menjadi 16000

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/709655cbfa2cb0b3c81672bba7b38c8e4b723fff99a6dcaaf4bc87449a9bb7a9/id_train_0/common_voice_id_25361388.mp3', 'array': array([-2.32830644e-09,  1.86264515e-09, -9.31322575e-10, ...,
       -2.64349492e-05, -7.05649245e-06,  2.22347239e-06]), 'sampling_rate': 16000}, 'sentence': 'Saya mendengarkan cerita membosankan dari teman saya.', 'variant': ''}


In [ ]:
def prepare_dataset(batch):
    '''
    Fungsi ini melakukan praposes dataset, ingat bahwa ada dua komponen yang perlu dipraproses.
    - input_features adalah array audio (stereo) yang akan diproses oleh feature_extractor untuk menjadi spektogram.
    - labels adalah transkripsi yang akan diproses oleh tokenizer dengan menggunakan BPE untuk menjadi representasi numeriknya.
    '''
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/8310 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3643 [00:00<?, ? examples/s]

Kita mendefinisikan kelas yang memersiapkan batch data bernama `DataCollatorSpeechSeq2SeqWithPadding` untuk model kita. Tujuannya adalah memadukan (collate) data dan mengatur paddingnya agar dapat diproses oleh model. Ada beberapa hal yang perlu dicatat:

- Fungsi `__call__` dipanggil ketika objek kelas ini dioperasikan seperti sebuah fungsi. Fungsi ini mengambil sebuah list berisi dictionary, dengan setiap dictionary mewakili sebuah sampel yang memiliki `input_features` untuk fitur audio dan `labels` untuk teks target.

- `input_features` dikumpulkan dari semua sampel dan kemudian di-pad menggunakan metode `pad` dari `feature_extractor` yang terdapat dalam `processor`. Proses padding ini memastikan semua input memiliki panjang yang seragam, yang diperlukan untuk pemrosesan batch yang efisien.

- Sementara itu, label juga dipad untuk memastikan konsistensi dalam panjangnya, menggunakan metode `pad` dari `tokenizer` yang merupakan bagian dari `processor`. Area yang tidak tercakup oleh mask dipenuhi dengan nilai `-100`, yang menandakan bahwa token tersebut harus diabaikan dalam perhitungan loss.

- Sebelum proses inferensi, ada pemeriksaan untuk memastikan bahwa setiap baris label mulai dengan token awal (`bos_token_id`). Jika ya, token ini dihapus untuk mungkin menghindari pengaruhnya terhadap prediksi model.

- Sebuah dictionary yang berisi `input_features` dan `labels` yang telah diproses disiapkan untuk dikembalikan, siap untuk digunakan dalam pelatihan model.

Modul ini akan kita gunakan bersama dengan modul `Seq2SeqTrainer` oleh Huggingface, ini akan sangat mempermudah kita untuk melakukan training model tanpa harus mendefinisikan fungsi trainer dan tester serta evaluatornya.

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
# Kita definisikan modul ini sebagai data_collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
# Kita definisikan evaluator kita, atur sebagai WER
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    '''
    Fungsi ini akan mempermudah kita melakukan perhitungan WER setelah output dari inferensi model diberikan.
    Kita perlu untuk mendecode output dari model menjadi bentuk awalnya, dan pastikan agar kita tidak memasukan
    special tokens dengan mengatur skip_special_tokens menjadi True.
    '''
    pred_ids, label_ids = pred.predictions, pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small") # Load model Whisper Small

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "indonesian" # Pastikan untuk mengatur modenya sebagai bahasa Indonesia
model.generation_config.task = "transcribe" # Task transcribe untuk transkripsi ucapan

model.generation_config.forced_decoder_ids = None # Menunjukkan bahwa tidak ada token atau ID dekoder spesifik yang dipaksa untuk digunakan, memungkinkan model untuk secara bebas memilih output berdasarkan pelatihannya.

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-id",  # Ubah nama repository huggingface sesuai keinginan Anda untuk menguploadnya pada saat training selesai!
    per_device_train_batch_size=2, # Jika GPU anda unggul, Anda bisa mengubah nilai ini menjadi 16 untuk mempercepat proses training. Jika tidak, 2 sudah cukup optimal (Mengambil sekitar 5-6 GB GPU RAM)
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Mulai Training!

Proses training ini memakan waktu sekitar 2-3 jam dengan konfigurasi yang kami telah tetapkan, silakan menunggu hasil trainingnya 😁

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.365600,0.312850,22.953101
2000,0.373600,0.283995,20.063089
3000,0.322100,0.260836,19.084288
4000,0.246900,0.241842,17.664796
5000,0.116700,0.239313,17.168437


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint

TrainOutput(global_step=5000, training_loss=0.3811796499967575, metrics={'train_runtime': 10112.8962, 'train_samples_per_second': 0.989, 'train_steps_per_second': 0.494, 'total_flos': 2.8858540032e+18, 'train_loss': 0.3811796499967575, 'epoch': 1.2})

Dapat kita lihat bahwa model yang telah kita fine-tune berhasil memberikan WER sebesar 17.16%. Sementara itu, model Whisper Small sebelum di fine-tune dapat memberikan performa WER sebesar 18.40%. Ini berarti kita telah berhasil meningkatkan performa model sebesar 1.24% 😁. Berikut merupakan laporan hasil evaluasi performa model dari [paper resmi](https://cdn.openai.com/papers/whisper.pdf) Whisper oleh OpenAI.


![image.png](https://drive.google.com/uc?id=1xq_Ar-HXD8tl168HY5UhdOJg46nvD1xr)


**Terima Kasih!**

## Referensi

Terima kasih atas beberapa referensi yang membantu kami membuat notebook ini:
- https://huggingface.co/blog/fine-tune-whisper
- https://github.com/openai/whisper/tree/main
- https://commonvoice.mozilla.org/
- https://huggingface.co/datasets/mozilla-foundation/common_voice_16_1